## Exploring Trey Hendrickons statistics from NFLVerse and Pro Football Focus

As of March 2024 the Cincinnati Bengals have allowed Trey Hendrickson to seek a trade elsewhere in the league. This comes soon after Maxx Crosby signed a [contract](https://www.nfl.com/news/raiders-sign-maxx-crosby-three-year-106-million-extension-highest-paid-non-qb) making him the highest paid non-QB. On paper does Trey have an argument that he should perhaps be paid more than that? 

Of course statistics can only capture so much. There are intangibles to a player and not all on field production can be tied to a statistic. ProFootballFocus takes those things into account and has ranked Trey as the 10th best DE in the league. Let's see what the stats say though.

In [4]:
import pandas as pd
import sys
sys.path.append('../..')

from utils import get_def_player_stats
df = get_def_player_stats(2024)
df.head()

,season,week,season_type,player_id,player_name,player_display_name,position,position_group,headshot_url,team,...,def_pass_defended,def_tds,def_fumbles,def_fumble_recovery_own,def_fumble_recovery_yards_own,def_fumble_recovery_opp,def_fumble_recovery_yards_opp,def_safety,def_penalty,def_penalty_yards
0,2024,1,REG,00-0023853,M.Prater,Matt Prater,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,ARI,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0
1,2024,5,REG,00-0026158,J.Flacco,Joe Flacco,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,JAX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
2,2024,1,REG,00-0026190,C.Campbell,Calais Campbell,DE,DL,https://static.www.nfl.com/image/upload/f_auto...,MIA,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
3,2024,2,REG,00-0026190,C.Campbell,Calais Campbell,DE,DL,https://static.www.nfl.com/image/upload/f_auto...,MIA,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
4,2024,3,REG,00-0026190,C.Campbell,Calais Campbell,DE,DL,https://static.www.nfl.com/image/upload/f_auto...,MIA,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0


In [60]:
TOP_TEN_DEs = [
    "Aidan Hutchinson",
    "Myles Garrett",
    "T.J. Watt",
    "Nick Bosa",
    "Khalil Mack",
    "Micah Parsons",
    "Jared Verse",
    "Alex Highsmith",
    "Will Anderson",
    "Trey Hendrickson",
]

top_10_df = (
    df[(df["season_type"] == "REG") & (df["player_display_name"].isin(TOP_TEN_DEs))]
    .groupby(["player_id"])
    .agg(
        {
            "player_display_name": "max",
            "week": "count",
            "team": "max",
            "def_tackles": "sum",
            "def_tackles_for_loss": "sum",
            "def_tackles_for_loss_yards": "sum",
            "def_sacks": "sum",
            "def_qb_hits": "sum",
        }
    )
)

# Rank the raw categories
top_10_df["def_tackle_rank"] = top_10_df["def_tackles"].rank(ascending=False)
top_10_df["def_tackles_for_loss_rank"] = top_10_df["def_tackles_for_loss"].rank(
    ascending=False
)
top_10_df["def_sacks_rank"] = top_10_df["def_sacks"].rank(ascending=False)
top_10_df["def_qb_hits_rank"] = top_10_df["def_qb_hits"].rank(ascending=False)
top_10_df["def_avg_rank"] = top_10_df[
    [
        "def_tackle_rank",
        "def_tackles_for_loss_rank",
        "def_sacks_rank",
        "def_qb_hits_rank",
    ]
].mean(axis=1)
top_10_df[
    [
        "player_display_name",
        "def_tackle_rank",
        "def_tackles_for_loss_rank",
        "def_sacks_rank",
        "def_qb_hits_rank",
        "def_avg_rank",
    ]
]

,player_display_name,def_tackle_rank,def_tackles_for_loss_rank,def_sacks_rank,def_qb_hits_rank,def_avg_rank
player_id,,,,,,
00-0031040,Khalil Mack,9.0,10.0,8.5,9.0,9.125
00-0033868,Myles Garrett,1.5,1.0,2.0,2.0,1.625
00-0033886,T.J. Watt,1.5,2.5,4.0,3.0,2.750
00-0033935,Trey Hendrickson,4.5,2.5,1.0,1.0,2.250
00-0035717,Nick Bosa,4.5,5.0,6.0,4.0,4.875
00-0036333,Alex Highsmith,7.0,8.0,8.5,10.0,8.375
00-0036932,Micah Parsons,6.0,6.0,3.0,5.0,5.000
00-0037236,Aidan Hutchinson,10.0,9.0,7.0,8.0,8.500
00-0039108,Will Anderson,8.0,4.0,5.0,6.0,5.750


In [ ]:
# Try a Pandas Styled Table
from utils import style_rankings

colnames = {
    "player_display_name": "Player",
    "def_tackle_rank": "Tackles",
    "def_tackles_for_loss_rank": "Tackles For Loss",
    "def_sacks_rank": "Sacks",
    "def_qb_hits_rank": "QB Hits",
}
top_10_df.sort_values("def_avg_rank")[
    [
        "player_display_name",
        "def_tackle_rank",
        "def_tackles_for_loss_rank",
        "def_sacks_rank",
        "def_qb_hits_rank",
    ]
].style.pipe(style_rankings, "Defensive End Rankings 2024", colnames)

Player,Tackles,Tackles For Loss,Sacks,QB Hits
Myles Garrett,1.5,1.0,2.0,2.0
Trey Hendrickson,4.5,2.5,1.0,1.0
T.J. Watt,1.5,2.5,4.0,3.0
Nick Bosa,4.5,5.0,6.0,4.0
Micah Parsons,6.0,6.0,3.0,5.0
Will Anderson,8.0,4.0,5.0,6.0
Jared Verse,3.0,7.0,10.0,7.0
Alex Highsmith,7.0,8.0,8.5,10.0
Aidan Hutchinson,10.0,9.0,7.0,8.0
Khalil Mack,9.0,10.0,8.5,9.0


Trey Hendrickson is definitely towards the top of the pack especially in the passing game which is consistent with what we've seen all year. Not everyone in this list played all season, though. Aidan Hutchinson for example was injured in week 6 and was the top rated DE by PFF. 

In [68]:
# Let's see how many weeks each player played in 2024
top_10_df[['player_display_name', 'team', 'week']]

,player_display_name,team,week
player_id,,,
00-0037236,Aidan Hutchinson,DET,5
00-0036333,Alex Highsmith,PIT,11
00-0039852,Jared Verse,LA,17
00-0031040,Khalil Mack,LAC,15
00-0036932,Micah Parsons,DAL,13
00-0033868,Myles Garrett,CLE,15
00-0035717,Nick Bosa,SF,13
00-0033886,T.J. Watt,PIT,16
00-0033935,Trey Hendrickson,CIN,17


We could divide the statistics by number of weeks, but PFF provides the snap count by pass, rush, and special teams plays. So let's use those instead.

In [ ]:
NUMBER_OF_SNAPS = {
    "Aidan Hutchinson": {"pass": 198, "rush": 77},
    "Alex Highsmith": {"pass": 321, "rush": 237},
    "Jared Verse": {"pass": 546, "rush": 356},
    "Khalil Mack": {"pass": 419, "rush": 232},
    "Micah Parsons": {"pass": 427, "rush": 243},
    "Myles Garrett": {"pass": 477, "rush": 328},
    "Nick Bosa": {"pass": 420, "rush": 267},
    "T.J. Watt": {"pass": 599, "rush": 370},
    "Trey Hendrickson": {"pass": 514, "rush": 273},
    "Will Anderson": {"pass": 418, "rush": 217},
}

top_10_df = top_10_df.sort_values("player_display_name")
top_10_df["pass_snaps"] = [v.get("pass") for _, v in NUMBER_OF_SNAPS.items()]
top_10_df["rush_snaps"] = [v.get("rush") for _, v in NUMBER_OF_SNAPS.items()]

# Calculate the stats per snap
# Tackles and TFL are classified as a running play
# Sacks and QB Hits are assumed to be a passing play
# Coverage snaps were not counted as DEs are mostly not a coverage position though some schemes do use them that way
# (All snap data is from PFF)
top_10_df["def_tackles_ps"] = top_10_df["def_tackles"] / top_10_df["rush_snaps"]
top_10_df["def_tackles_for_loss_ps"] = (
    top_10_df["def_tackles_for_loss"] / top_10_df["rush_snaps"]
)
top_10_df["def_sacks_ps"] = top_10_df["def_sacks"] / top_10_df["pass_snaps"]
top_10_df["def_qb_hits_ps"] = top_10_df["def_qb_hits"] / top_10_df["pass_snaps"]

# Calculate the ranks for each ps stat
top_10_df["def_tackles_ps_rank"] = top_10_df["def_tackles_ps"].rank(ascending=False)
top_10_df["def_tackles_for_loss_ps_rank"] = top_10_df["def_tackles_for_loss_ps"].rank(
    ascending=False
)
top_10_df["def_sacks_ps_rank"] = top_10_df["def_sacks_ps"].rank(ascending=False)
top_10_df["def_qb_hits_ps_rank"] = top_10_df["def_qb_hits_ps"].rank(ascending=False)

# Add the average ranking column for sorting
top_10_df["def_avg_ps_rank"] = top_10_df[
    [
        "def_tackles_ps_rank",
        "def_tackles_for_loss_ps_rank",
        "def_sacks_ps_rank",
        "def_qb_hits_ps_rank",
    ]
].mean(axis=1)

Now let's take a look at rankings table using per snap statistics.

In [ ]:
colnames = {
    "player_display_name": "Player",
    "def_tackles_ps_rank": "Tackles",
    "def_tackles_for_loss_ps_rank": "Tackles For Loss",
    "def_sacks_ps_rank": "Sacks",
    "def_qb_hits_ps_rank": "QB Hits",
}
top_10_df.sort_values("def_avg_ps_rank")[
    [
        "player_display_name",
        "def_tackles_ps_rank",
        "def_tackles_for_loss_ps_rank",
        "def_sacks_ps_rank",
        "def_qb_hits_ps_rank",
    ]
].style.pipe(style_rankings, "Defensive End Rank Per Snap 2024", colnames)

Player,Tackles,Tackles For Loss,Sacks,QB Hits
Aidan Hutchinson,1.0,1.0,1.0,1.0
Trey Hendrickson,6.0,3.0,2.0,2.0
Myles Garrett,5.0,4.0,3.0,3.0
Will Anderson,2.0,2.0,5.0,6.0
Nick Bosa,3.0,5.0,6.0,4.0
Micah Parsons,4.0,7.0,4.0,5.0
T.J. Watt,8.0,6.0,7.0,7.0
Alex Highsmith,7.0,8.0,8.0,8.0
Jared Verse,9.0,9.0,10.0,10.0
Khalil Mack,10.0,10.0,9.0,9.0


Trey Hendrickson still ranks near the top of the table based on 2024 per snap statistics. Based on statistics alone it might make sense to pay Trey as a top defensive end, but there are many other considerations that we could explore:

- How much if any does defensive end production tend to fall off at Trey's age.
- Some have claimed that if Trey had more of a peer on the other side of the line that he'd have even more sacks. Is there a way to assess that with the statistics available?
- Does Trey tend to face easier/tougher competition than other ends in the league?